<h1 style="text-align: center;"> Concepte și Aplicații în Vederea Artificială - Tema 1 </h1>
<h1 style="text-align: center;"> Calculator automat de scor pentru jocul Mathable </h1>

<p style="text-align: right;"> Proiect realizat de: Bucă Mihnea-Vicențiu <p>

**Obiectiv**

Scopul acestui proiect este dezvoltarea unui sistem automat pentru a calcula scorul în jocul Mathable.

### Biblioteci

In [1]:
import os 
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import matplotlib

print("python==", os.sys.version, sep="")
print("opencv_python==", cv.__version__, sep="")
print("numpy==", np.__version__, sep="")
print("matplotlib==", matplotlib.__version__, sep="")

python==3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
opencv_python==4.10.0
numpy==2.1.2
matplotlib==3.9.2


### Evaluare

In [4]:
# Folders and files we use
folder_path_input = '../testare'
folder_path_output = '../352_Buca_Mihnea-Vicentiu'

# number of games (add +1 since the loop starts from 1)
nr_games = 4 + 1
nr_turns = 50 + 1

### Task 1.

In [5]:
def show_img(title, img):
    plt.figure(figsize = (10, 10))
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [6]:
def extract_table(image, debug = False):
    # find the coordinates of the table
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image_gray, 5)
    image_g_blur = cv.GaussianBlur(image_m_blur, (5, 5), 0) 
     
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(image_g_blur, kernel, iterations=2)
        
    edges =  cv.Canny(thresh, 150, 400)
    edges = cv.dilate(edges, kernel, iterations=2)

    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # find bounding box of the table
    top_left     = (float('inf'), float('inf'))
    top_right    = (-float('inf'), float('inf'))
    bottom_left  = (float('inf'), -float('inf'))
    bottom_right = (-float('inf'), -float('inf'))

    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        top_left = (min(top_left[0], x), min(top_left[1], y))
        top_right = (max(top_right[0], x + w), min(top_right[1], y))
        bottom_left = (min(bottom_left[0], x), max(bottom_left[1], y + h))
        bottom_right = (max(bottom_right[0], x + w), max(bottom_right[1], y + h)) 
    
    if debug:
        image_copy = image.copy()
        cv.circle(image_copy, tuple(top_left), 20, (0, 0, 255), -1)
        cv.circle(image_copy, tuple(top_right), 20, (0, 0, 255), -1)
        cv.circle(image_copy, tuple(bottom_left), 20, (0, 0, 255) ,-1)
        cv.circle(image_copy, tuple(bottom_right), 20, (0, 0, 255), -1)
        show_img("Bounding Box", image_copy)

    # transform the perspective of the image
    width, height = 910, 910 
    bounding_box = np.array([top_left, top_right, bottom_left, bottom_right], dtype="float32")
    destination_points = np.array([[0, 0], [width, 0], [0, height], [width, height]], dtype="float32")
    M = cv.getPerspectiveTransform(bounding_box, destination_points)
    result = cv.warpPerspective(image, M, (width, height))

    if debug:
        show_img("Extracted Table", result)
    
    return result

# Test the function to see if we extract the correct bounding box
img = cv.imread('../imagini_auxiliare/04.jpg')
table = extract_table(img, debug=False)

In [7]:
def get_horizontal_vertical(): 
    lines_horizontal = []
    for i in range(0, 911, 65):
        l = []
        l.append((0, i))
        l.append((909, i))
        lines_horizontal.append(l)

    lines_vertical = []
    for i in range(0, 911, 65):
        l = []
        l.append((i, 0))
        l.append((i, 909))
        lines_vertical.append(l)
    
    return lines_horizontal, lines_vertical

def show_horizontal_vertical(lines_horizontal, lines_vertical, debug = False):
    img = cv.imread('../imagini_auxiliare/03.jpg')
    result = extract_table(img)
    for line in lines_vertical: 
        cv.line(result, line[0], line[1], (0, 255, 0), 4)
        for line in lines_horizontal: 
            cv.line(result, line[0], line[1], (0, 0, 255), 4)

    if debug:
        show_img('Horizontal/Vertical Lines', result)

lines_horizontal, lines_vertical = get_horizontal_vertical()
show_horizontal_vertical(lines_horizontal, lines_vertical, debug=False)

In [8]:
def extract_color_of_each_cell(): # Hardcoded the colors of the game grid
    table = [
        ["Y", "W", "W", "W", "W", "W", "Y", "Y", "W", "W", "W", "W", "W", "Y"],
        ["W", "V", "W", "W", "B", "W", "W", "W", "W", "B", "W", "W", "V", "W"],
        ["W", "W", "V", "W", "W", "B", "W", "W", "B", "W", "W", "V", "W", "W"],
        ["W", "W", "W", "V", "W", "W", "B", "B", "W", "W", "V", "W", "W", "W"],
        ["W", "B", "W", "W", "V", "W", "B", "B", "W", "V", "W", "W", "B", "W"],
        ["W", "W", "B", "W", "W", "W", "W", "W", "W", "W", "W", "B", "W", "W"],
        ["Y", "W", "W", "B", "B", "W", "Y", "Y", "W", "B", "B", "W", "W", "Y"],
        ["Y", "W", "W", "B", "B", "W", "Y", "Y", "W", "B", "B", "W", "W", "Y"],
        ["W", "W", "B", "W", "W", "W", "W", "W", "W", "W", "W", "B", "W", "W"],
        ["W", "B", "W", "W", "V", "W", "B", "B", "W", "V", "W", "W", "B", "W"],
        ["W", "W", "W", "V", "W", "W", "B", "B", "W", "W", "V", "W", "W", "W"],
        ["W", "W", "V", "W", "W", "B", "W", "W", "B", "W", "W", "V", "W", "W"],
        ["W", "V", "W", "W", "B", "W", "W", "W", "W", "B", "W", "W", "V", "W"],
        ["Y", "W", "W", "W", "W", "W", "Y", "Y", "W", "W", "W", "W", "W", "Y"],
    ] 

    return table

In [9]:
# Needed variables
visited_positions = set()
current_possible_positions = set()
table_colors = extract_color_of_each_cell()
table_moves_i, table_moves_j = [-1, 1, 0, 0], [0, 0, -1, 1]

def reset_visited_positions():
    visited_positions = set(((6, 6), (6, 7), (7, 6), (7, 7)))
    current_possible_positions = set(((6, 5), (7, 5), (6, 8), (7, 8), (5, 6), (5, 7), (8, 6), (8, 7)))
    return visited_positions, current_possible_positions

In [10]:
def check_position(i, j):
    if i < 0 or i >= 14 or j < 0 or j >= 14:
        return False

    if (i, j) in visited_positions:
        return False
    
    return True

def format_position(position):
    return "{}{}".format(position[1] + 1, chr(position[0] + 65))

In [11]:
def compare_cells(thresh, test_img, i, j):
    const_add = 10

    # take the patch from the thresholded image
    y_min = lines_vertical[j][0][0] + const_add
    y_max = lines_vertical[j + 1][1][0] - const_add
    x_min = lines_horizontal[i][0][1] + const_add
    x_max = lines_horizontal[i + 1][1][1] - const_add

    patch = thresh[y_min:y_max, x_min:x_max].copy()
    patch_test = test_img[y_min:y_max, x_min:x_max].copy()

    score = np.sum((patch - patch_test) ** 2)

    if score > 1000:
        return True

    return False

def extract_cells(thresh, test_img):
    thresh_table_hsv = []
    thresh_hsv = cv.cvtColor(thresh, cv.COLOR_BGR2HSV)

    test_table_hsv = []
    test_hsv = cv.cvtColor(test_img, cv.COLOR_BGR2HSV)

    # apply hsv thresholding for each color in grid
    for color in ["W", "Y", "B", "V"]:
        if color == "W":
            low_yellow = np.array([90, 0, 175])
            high_yellow = np.array([100, 125, 255])
        elif color == "Y":
            low_yellow = np.array([0, 75, 0])
            high_yellow = np.array([90, 231, 255])
        elif color == "B":
            low_yellow = np.array([90, 210, 0])
            high_yellow = np.array([255, 255, 255])
        elif color == "V":
            low_yellow = np.array([110, 0, 0])
            high_yellow = np.array([255, 255, 255])

        thresh_table_hsv.append(cv.inRange(thresh_hsv, low_yellow, high_yellow))
        test_table_hsv.append(cv.inRange(test_hsv, low_yellow, high_yellow))

    # walk through the grid and compare only with viable cells 
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            
            # check if the cell is occupied
            if (j, i) in visited_positions:
                continue

            # check if the cell is viable 
            if (j, i) not in current_possible_positions:
                continue

            color = table_colors[j][i]

            if color == "W":
                img1 = thresh_table_hsv[0]
                img2 = test_table_hsv[0]
            elif color == "Y":
                img1 = thresh_table_hsv[1]
                img2 = test_table_hsv[1]
            elif color == "B":
                img1 = thresh_table_hsv[2]
                img2 = test_table_hsv[2]
            elif color == "V":
                img1 = thresh_table_hsv[3]
                img2 = test_table_hsv[3]

            res = compare_cells(img1, img2, i, j)
            
            if res is True:
                # mark the cell as visited
                visited_positions.add((j, i))
                current_possible_positions.remove((j, i))

                # expand the possible positions
                for idx in range(len(table_moves_i)):
                    add_i = table_moves_i[idx]
                    add_j = table_moves_j[idx]
                    if check_position(j + add_j, i + add_i):
                        current_possible_positions.add((j + add_j, i + add_i))
                      
                return format_position((i, j))    
    
    # should never reach this point
    return None

In [12]:
task1_moves = []

# extract the first image for the first comparison
first_img = cv.imread(f'../imagini_auxiliare/01.jpg')
first_img = extract_table(first_img)

for i in range(1, nr_games):
    # reset the visited positions and the possible positions
    visited_positions, current_possible_positions = reset_visited_positions()
    org = first_img

    list_moves = []
    for j in range(1, nr_turns):
        if j < 10: img = cv.imread(f'{folder_path_input}/{i}_0{j}.jpg')
        else: img = cv.imread(f'{folder_path_input}/{i}_{j}.jpg')

        result = extract_table(img)
        res = extract_cells(org, result)

        list_moves += [res]
        org = result 

    # append the moves for the current task 
    task1_moves.append(list_moves.copy())

# for row in task1_moves:
#     print(row)

### Task 2 

In [13]:
mathable_digits = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
    '20', '21', '24', '25', '27', '28', 
    '30', '32', '35', '36',
    '40', '42', '45', '48', '49',
    '50', '54', '56',
    '60', '63', '64',
    '70', '72',
    '80', '81',
    '90'
];

cnt = {}
for digit in mathable_digits:
    if digit not in cnt:
        cnt[digit] = 0

In [14]:
def find_bounding_box(img):
    image_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image_gray, 5)
    image_g_blur = cv.GaussianBlur(image_m_blur, (5, 5), 0) 
     
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(image_g_blur, kernel, iterations=2)
        
    edges =  cv.Canny(thresh, 250, 400)
    edges = cv.dilate(edges, kernel, iterations=2)

    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    max_contour = max(contours, key=cv.contourArea)
    x, y, w, h = cv.boundingRect(max_contour)
    
    img = img[y:y+h+1, x:x+w+1].copy()
    return img

def find_bounding_box_2(img):
    image_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    thresh = cv.Canny(image_gray, 100, 400)

    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    x_min, x_max, y_min, y_max = float('inf'), -float('inf'), float('inf'), -float('inf')
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        x_min = min(x_min, x)
        x_max = max(x_max, x + w)
        y_min = min(y_min, y)
        y_max = max(y_max, y + h)

    res = img[y_min:y_max, x_min:x_max].copy()
    return res

In [15]:
def extract_portion(img, i, j):
    const_add = 5
    y_min = lines_vertical[j][0][0] + const_add
    y_max = lines_vertical[j + 1][1][0] - const_add
    x_min = lines_horizontal[i][0][1] + const_add
    x_max = lines_horizontal[i + 1][1][1] - const_add

    patch = img[y_min:y_max, x_min:x_max].copy()
    patch = find_bounding_box(patch)
    patch = find_bounding_box_2(patch)
    return patch

def extract_coordinates(move):
    y = 0
    while move[0].isdigit():
        y = y * 10 + int(move[0])
        move = move[1:]

    y -= 1
    x = ord(move[0]) - ord('A')
    return y, x

In [16]:
def extract_data_from_auxiliary_images():
    idx = 0
    img = cv.imread('../imagini_auxiliare/03.jpg')
    img = extract_table(img)
    for j in range(5, 11):
        for i in range(4, 12 - (0 if j < 10 else 2)):
            patch = extract_portion(img, i, j) 

            output_dir = f'cifre/{mathable_digits[idx]}'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir) 
 
            label_img = f'{cnt[mathable_digits[idx]]}' if cnt[mathable_digits[idx]] >= 10 else f'0{cnt[mathable_digits[idx]]}'
            cv.imwrite(f'cifre/{mathable_digits[idx]}/{label_img}.jpg', patch)

            cnt[mathable_digits[idx]] += 1
            idx += 1 

    idx = 0
    img = cv.imread('../imagini_auxiliare/04.jpg')
    img = extract_table(img)
    for j in range(0, 14, 2):
        for i in range(0, 14, 2):
            patch = extract_portion(img, i, j)
            
            output_dir = f'cifre/{mathable_digits[idx]}'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir) 
 
            label_img = f'{cnt[mathable_digits[idx]]}' if cnt[mathable_digits[idx]] >= 10 else f'0{cnt[mathable_digits[idx]]}'
            cv.imwrite(f'cifre/{mathable_digits[idx]}/{label_img}.jpg', patch)

            cnt[mathable_digits[idx]] += 1
            if mathable_digits[idx] == '90':
                break

            idx += 1

In [17]:
def extract_data_from_training_images():
    training_test_input = '../antrenare'
    training_test_output = '../antrenare'

    for i in range(1, 5):
        for j in range(1, 51):
            if j < 10: 
                img = cv.imread(f'{training_test_input}/{i}_0{j}.jpg')
                with open(f'{training_test_output}/{i}_0{j}.txt', 'r') as file:
                    output = file.readline().strip()
            else: 
                img = cv.imread(f'{training_test_input}/{i}_{j}.jpg')
                with open(f'{training_test_output}/{i}_{j}.txt', 'r') as file:
                    output = file.readline().strip() 
            
            first, second = output.split(' ')

            y, x = extract_coordinates(first)

            img = extract_table(img)
            patch = extract_portion(img, x, y) 

            output_dir = f'cifre/{second}'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir) 
            
            label_img = f'{cnt[second]}' if cnt[second] >= 10 else f'0{cnt[second]}'
            cv.imwrite(f'cifre/{second}/{label_img}.jpg', patch)
            cnt[second] += 1


In [18]:
def extract_data_from_fake_test():
    fake_test_input = '../evaluare/fake_test'
    fake_test_output = '../evaluare/cod_evaluare/fake_test_gt'

    for i in range(1, 2):
        for j in range(1, 51):
            if j < 10: 
                img = cv.imread(f'{fake_test_input}/{i}_0{j}.jpg')
                with open(f'{fake_test_output}/{i}_0{j}.txt', 'r') as file:
                    output = file.readline().strip()
            else: 
                img = cv.imread(f'{fake_test_input}/{i}_{j}.jpg')
                with open(f'{fake_test_output}/{i}_{j}.txt', 'r') as file:
                    output = file.readline().strip() 

            first, second = output.split(' ')

            y, x = extract_coordinates(first)

            img = extract_table(img)
            patch = extract_portion(img, x, y)

            output_dir = f'cifre/{second}'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            label_img = f'{cnt[second]}' if cnt[second] >= 10 else f'0{cnt[second]}'
            cv.imwrite(f'cifre/{second}/{label_img}.jpg', patch)
            cnt[second] += 1             

In [19]:
def get_digits():
    extract_data_from_auxiliary_images()
    extract_data_from_training_images()
    extract_data_from_fake_test()

In [20]:
def rotate_digits():
    lazy_add = []
    for digit in mathable_digits:
        digit_dir = f'cifre/{digit}'
        
        for filename in os.listdir(digit_dir):
            if filename.endswith('.jpg'):
                img = cv.imread(f'{digit_dir}/{filename}')

                for i in [5]:
                    image_center = tuple(np.array(img.shape[1::-1]) / 2)
                    rot_mat = cv.getRotationMatrix2D(image_center, i, 1)
                    rotated = cv.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REPLICATE)

                    label_image = f'{cnt[digit]}' if cnt[digit] >= 10 else f'0{cnt[digit]}'

                    lazy_add += [(f'cifre/{digit}/{label_image}.jpg', rotated)]

                    cnt[digit] += 1
                
                for i in [5]:
                    image_center = tuple(np.array(img.shape[1::-1]) / 2)
                    rot_mat = cv.getRotationMatrix2D(image_center, -i, 1)
                    rotated = cv.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REPLICATE)

                    label_image = f'{cnt[digit]}' if cnt[digit] >= 10 else f'0{cnt[digit]}'
                    lazy_add += [(f'cifre/{digit}/{label_image}.jpg', rotated)]

                    cnt[digit] += 1
    
    return lazy_add 

In [21]:
def change_lumiosity_slightly():
    lazy_add = []
    for digit in mathable_digits:
        digit_dir = f'cifre/{digit}'
        
        for filename in os.listdir(digit_dir):
            if filename.endswith('.jpg'):
                img = cv.imread(f'{digit_dir}/{filename}')

                for alpha, beta in [(1.1, 10), (0.9, -10)]:
                    label_image = f'{cnt[digit]}' if cnt[digit] >= 10 else f'0{cnt[digit]}'
                    img_aug = cv.convertScaleAbs(img, alpha=alpha, beta=beta)
                    lazy_add.append((f'cifre/{digit}/{label_image}.jpg', img_aug))
                    cnt[digit] += 1                
    
    return lazy_add 

In [22]:
def add_noise():
    lazy_add = []
    for digit in mathable_digits:
        digit_dir = f'cifre/{digit}'
        
        for filename in os.listdir(digit_dir):
            if filename.endswith('.jpg'):
                img = cv.imread(f'{digit_dir}/{filename}')

                label_image = f'{cnt[digit]}' if cnt[digit] >= 10 else f'0{cnt[digit]}'
                img_aug = cv.add(img, np.random.normal(0, 10, img.shape), dtype=cv.CV_8UC3)
                lazy_add.append((f'cifre/{digit}/{label_image}.jpg', img_aug))
                cnt[digit] += 1                
    
    return lazy_add

In [23]:
def clear_folders():
    for digit in mathable_digits:
        digit_dir = f'cifre/{digit}'
        for filename in os.listdir(digit_dir):
            if filename.endswith('.jpg'):
                os.remove(f'{digit_dir}/{filename}')
        os.removedirs(digit_dir)
    
    for digit in mathable_digits:
        cnt[digit] = 0

In [24]:
def extract_dataset():
    clear_folders()
    get_digits()
    rotations = rotate_digits()
    brighness = change_lumiosity_slightly()
    noise = add_noise()

    for filename, img in rotations:
        cv.imwrite(filename, img)
    
    for filename, img in brighness:
        cv.imwrite(filename, img)
    
    for filename, img in noise:
        cv.imwrite(filename, img)

# extract_dataset()

In [25]:
def extract_table_information():
    table = [
        ["Y", "W", "W", "W", "W", "W", "Y", "Y", "W", "W", "W", "W", "W", "Y"],
        ["W", "V", "W", "W", "/", "W", "W", "W", "W", "/", "W", "W", "V", "W"],
        ["W", "W", "V", "W", "W", "-", "W", "W", "-", "W", "W", "V", "W", "W"],
        ["W", "W", "W", "V", "W", "W", "+", "*", "W", "W", "V", "W", "W", "W"],
        ["W", "/", "W", "W", "V", "W", "*", "+", "W", "V", "W", "W", "/", "W"],
        ["W", "W", "-", "W", "W", "W", "W", "W", "W", "W", "W", "-", "W", "W"],
        ["Y", "W", "W", "*", "+", "W", "k", "Y", "W", "*", "+", "W", "W", "Y"],
        ["Y", "W", "W", "+", "*", "W", "Y", "Y", "W", "+", "*", "W", "W", "Y"],
        ["W", "W", "-", "W", "W", "W", "W", "W", "W", "W", "W", "-", "W", "W"],
        ["W", "/", "W", "W", "V", "W", "+", "*", "W", "V", "W", "W", "/", "W"],
        ["W", "W", "W", "V", "W", "W", "*", "+", "W", "W", "V", "W", "W", "W"],
        ["W", "W", "V", "W", "W", "-", "W", "W", "-", "W", "W", "V", "W", "W"],
        ["W", "V", "W", "W", "/", "W", "W", "W", "W", "/", "W", "W", "V", "W"],
        ["Y", "W", "W", "W", "W", "W", "Y", "Y", "W", "W", "W", "W", "W", "Y"],
    ]

    placements = [[0 for _ in range(14)] for _ in range(14)]
    placements[6][6] = 1
    placements[6][7] = 2
    placements[7][6] = 3
    placements[7][7] = 4

    return table, placements

In [26]:
# Needed variables
remaining_digits = {}
visited_positions = set()
table_information, table_placements = extract_table_information()

def get_number_on_each_cell():
    visited_positions = set(((6, 6), (6, 7), (7, 6), (7, 7)))
    return visited_positions

def get_number_for_each_digit():
    cnt = {}
    special_digits = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
    for digit in mathable_digits:
        if digit in special_digits:
            cnt[digit] = 7
        else:
            cnt[digit] = 1
    
    return cnt

In [27]:
def given_numbers_op(mn, mx, curr_candidate, op):
    if op == '/':
        return True if mn != 0 and mx % mn == 0 and mx // mn == curr_candidate else False
    if op == '-':
        return True if mx - mn == curr_candidate else False
    if op == '+':
        return True if mx + mn == curr_candidate else False
    if op == '*':
        return True if mx * mn == curr_candidate else False
    return False

def check_operation(x, y, curr_candidate, direction, op):
    if direction == 'left':
        if (y, x - 1) in visited_positions and (y, x - 2) in visited_positions:
            mx = max(table_placements[y][x - 1], table_placements[y][x - 2])
            mn = min(table_placements[y][x - 1], table_placements[y][x - 2])
            return given_numbers_op(mn, mx, curr_candidate, op) 
    
    if direction == 'right':
        if (y, x + 1) in visited_positions and (y, x + 2) in visited_positions:
            mx = max(table_placements[y][x + 1], table_placements[y][x + 2])
            mn = min(table_placements[y][x + 1], table_placements[y][x + 2])
            return given_numbers_op(mn, mx, curr_candidate, op)
    
    if direction == 'down':
        if (y + 1, x) in visited_positions and (y + 2, x) in visited_positions:
            mx = max(table_placements[y + 1][x], table_placements[y + 2][x])
            mn = min(table_placements[y + 1][x], table_placements[y + 2][x])
            return given_numbers_op(mn, mx, curr_candidate, op)
    
    if direction == 'up':
        if (y - 1, x) in visited_positions and (y - 2, x) in visited_positions:
            mx = max(table_placements[y - 1][x], table_placements[y - 2][x])
            mn = min(table_placements[y - 1][x], table_placements[y - 2][x])
            return given_numbers_op(mn, mx, curr_candidate, op)

    return False

def find_all_equation(x, y, curr_candidate):
    for special_sign in ['/', '-', '+', '*']:
        if table_information[y][x] == special_sign:
            for direction in ['left', 'right', 'down', 'up']:
                if check_operation(x, y, curr_candidate, direction, special_sign):
                    return True 
            return False
        

    for operation in ['/', '-', '+', '*']:
        for direction in ['left', 'right', 'down', 'up']:
            if check_operation(x, y, curr_candidate, direction, operation):
                return True
    
    return False

In [28]:
def transform_to_gray(img):
    thresh = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(thresh, 100, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    return thresh

In [29]:
def find_best_matching(test_img, y, x):
    best_match = None
    best_score = np.inf 

    patch = extract_portion(test_img, x, y)
    patch = transform_to_gray(patch)

    for digit in mathable_digits:
        if remaining_digits[digit] == 0:
            continue
        
        if find_all_equation(x, y, int(digit)) is False:
            continue

        digit_dir = f'cifre/{digit}'
        for filename in os.listdir(digit_dir):
            if filename.endswith('.jpg'):
                img = cv.imread(f'{digit_dir}/{filename}')
                patch = extract_portion(test_img, x, y)

                img = cv.resize(img, (patch.shape[1], patch.shape[0]), interpolation=cv.INTER_LINEAR)

                img = transform_to_gray(img)
                patch = transform_to_gray(patch)

                score = cv.matchTemplate(patch, img, cv.TM_SQDIFF_NORMED)
                score = np.max(score)

                if score < best_score:
                    best_score = score
                    best_match = digit
    
    return best_match


In [30]:
task2_digits = [] # store the digits for each move made in the second task

for i in range(1, nr_games):
    visited_positions = get_number_on_each_cell()
    remaining_digits = get_number_for_each_digit()
    _, table_placements = extract_table_information()

    list_digits = []
    for j in range(1, nr_turns):
        if j < 10: img = cv.imread(f'{folder_path_input}/{i}_0{j}.jpg')
        else: img = cv.imread(f'{folder_path_input}/{i}_{j}.jpg')

        current_position = task1_moves[i - 1][j - 1]

        y, x = extract_coordinates(current_position)
        
        img = extract_table(img)
        res = find_best_matching(img, y, x)

        visited_positions.add((y, x))
        table_placements[y][x] = int(res)
        remaining_digits[res] -= 1

        list_digits += [res]
    
    task2_digits.append(list_digits.copy())

for row in task2_digits:
    print(row)

['2', '6', '7', '1', '8', '15', '14', '11', '2', '13', '25', '40', '7', '24', '5', '30', '10', '20', '12', '2', '7', '19', '18', '9', '9', '81', '4', '6', '36', '42', '2', '8', '1', '35', '70', '28', '4', '16', '10', '21', '2', '1', '3', '5', '6', '1', '4', '2', '7', '7']
['2', '3', '6', '7', '9', '7', '3', '11', '16', '25', '9', '6', '4', '10', '6', '17', '28', '27', '9', '54', '6', '9', '5', '10', '1', '6', '4', '10', '13', '14', '8', '48', '15', '10', '5', '24', '9', '10', '21', '2', '7', '4', '10', '2', '6', '4', '2', '8', '2', '10']
['3', '7', '8', '10', '2', '80', '10', '4', '6', '10', '7', '3', '10', '10', '2', '7', '3', '3', '9', '5', '4', '7', '14', '4', '63', '1', '64', '54', '5', '49', '15', '4', '20', '27', '7', '24', '17', '7', '10', '11', '70', '7', '6', '9', '13', '3', '5', '30', '40', '10']
['4', '7', '28', '35', '63', '6', '1', '1', '2', '24', '20', '2', '4', '4', '8', '8', '7', '17', '2', '19', '10', '14', '16', '80', '2', '4', '4', '1', '3', '1', '81', '3', '18', '21

### Task 3

In [32]:
def find_all_eq_score(x, y, curr_candidate):
    cnt = 0
    for special_sign in ['/', '-', '+', '*']:
        if table_information[y][x] == special_sign:
            for direction in ['left', 'right', 'down', 'up']:
                if check_operation(x, y, curr_candidate, direction, special_sign):
                    cnt += 1
            return cnt * curr_candidate
    
    for direction in ['left', 'right', 'down', 'up']:
        for operation in ['/', '-', '+', '*']:
            if check_operation(x, y, curr_candidate, direction, operation):
                cnt += 1
                break

    multiplier = 1

    if table_information[y][x] == 'Y':
        multiplier = 3

    if table_information[y][x] == 'V':
        multiplier = 2
    
    return cnt * curr_candidate * multiplier

In [33]:
task3_score = []

for i in range(1, nr_games):
    _, table_placements = extract_table_information()
    visited_positions = get_number_on_each_cell()

    current_turns = []
    with open(f'{folder_path_input}/{i}_turns.txt', 'r') as file:
        while True:
            line = file.readline().strip()
            if not line:
                break
            moves = line.split(' ')
            current_turns += [int(moves[1])]

    current_turns.pop(0)
    current_turns += [51]

    list_turns = []

    sum_score = 0
    for j in range(1, nr_turns):
        if j < 10: img = cv.imread(f'{folder_path_input}/{i}_0{j}.jpg')
        else: img = cv.imread(f'{folder_path_input}/{i}_{j}.jpg')

        current_position = task1_moves[i - 1][j - 1]
        y, x = extract_coordinates(current_position)        

        current_number = task2_digits[i - 1][j - 1]

        res = find_all_eq_score(x, y, int(current_number))
        sum_score += res

        if current_turns[0] - 1 == j:
            list_turns += [sum_score]
            current_turns.pop(0)
            sum_score = 0

        visited_positions.add((y, x))
        table_placements[y][x] = int(current_number)

    task3_score.append(list_turns.copy())

### Output

In [34]:
if os.path.exists(folder_path_output) is False:
    os.makedirs(folder_path_output)

for i in range(1, nr_games):
    for j in range(1, nr_turns):
        file_output = f'{i}_{j}.txt' if j >= 10 else f'{i}_0{j}.txt'
        with open(f'{folder_path_output}/{file_output}', 'w') as file:
            file.write(f'{task1_moves[i - 1][j - 1]} {task2_digits[i - 1][j - 1]}')
            file.close()

    input_file = open(f'{folder_path_input}/{i}_turns.txt', 'r')
    
    idx = 0
    answer_task3 = []
    for line in input_file:
        y, x = line.strip().split(' ')
        answer_task3 += [(y, x, task3_score[i - 1][idx])]
        idx += 1
    
    with open(f'{folder_path_output}/{i}_scores.txt', 'w') as file:
        idx = 0
        L = len(answer_task3)
        for y, x, score in answer_task3:
            file.write(f'{y} {x} {score}')
            if idx != L - 1:
                file.write('\n')
            idx += 1

### Debug (Ignore)

In [135]:
import shutil

def get_files_into_testare():
    traning_data_set = '../antrenare'
    for i in range(1, 5):
        for j in range(1, 51):
            if j < 10: file = f'{i}_0{j}.jpg'
            else: file = f'{i}_{j}.jpg'

            shutil.copy(f'{traning_data_set}/{file}', f'../testare/{file}')
        
        shutil.copy(f'{traning_data_set}/{i}_turns.txt', f'../testare/{i}_turns.txt')

def get_file_gt():
    traning_data_set = '../antrenare'
    for i in range(1, 5):
        for j in range(1, 51):
            if j < 10: file = f'{i}_0{j}.txt'
            else: file = f'{i}_{j}.txt'

            shutil.copy(f'{traning_data_set}/{file}', f'../test_gt/{file}')
        
        shutil.copy(f'{traning_data_set}/{i}_scores.txt', f'../test_gt/{i}_scores.txt')

def clear_files():
    # folder_del = '../testare'
    # for filename in os.listdir(folder_del):
    #     os.remove(f'{folder_del}/{filename}')
    
    folder_del = '../test_gt'
    for filename in os.listdir(folder_del):
        os.remove(f'{folder_del}/{filename}')


# get_files_into_testare()
# get_file_gt()
# clear_files()